# Helpful Links
Tutorial provided by Professor Chanin Nantasenamat. 

[Data Professor YouTube Channel](https://www.youtube.com/dataprofessor/)

[Link to YouTube Tutorial](https://www.youtube.com/watch?v=jBlTQjcKuaY&t=640s)


*First modified: 21 May 2022*

*Last modified:* 	

## Installing libraries
Install the ChEMBL web service package so data can be retrieved from the database.

Set up a venv on Windows + VSCode using [this tutorial](https://www.youtube.com/watch?v=fBBAGXjg2fk) by [42technoman](https://www.youtube.com/c/42technoman).

        # initialise a virtual environment
        python -m venv newname
        # cd into the new directory
        # launch vscode 
        code .
        # activate the venv
        # cd into Scripts
        ./activate

Run the following command in the terminal:

        pip3 install chembl_webresource_client
        pip3 install pandas

In [ ]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client
import os 

: 

# Download Raw Data - Concise

In [4]:
df = pd.read_csv(os.getcwd()+"/data/bioactivity_preprocessed_data.csv")

In [5]:
df.head()

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity_class
0,CHEMBL187579,Cc1noc(C)c1CN1C(=O)C(=O)c2cc(C#N)ccc21,7200.0,intermediate
1,CHEMBL188487,O=C1C(=O)N(Cc2ccc(F)cc2Cl)c2ccc(I)cc21,9400.0,intermediate
2,CHEMBL185698,O=C1C(=O)N(CC2COc3ccccc3O2)c2ccc(I)cc21,13500.0,inactive
3,CHEMBL426082,O=C1C(=O)N(Cc2cc3ccccc3s2)c2ccccc21,13110.0,inactive
4,CHEMBL187717,O=C1C(=O)N(Cc2cc3ccccc3s2)c2c1cccc2[N+](=O)[O-],2000.0,intermediate
